Code to run SOCNN model on sample toy dataset.

In [1]:
import nnts
from nnts.utils import *

Working directory: 'C:\\Users\\mbinkowski\\cdsol-r-d.cluster\\cdsol-r-d.machine_learning_studies\\nntimeseries\\'


Using TensorFlow backend.


Defining sample data frame:
- column A enumerates entries, B contains random binomial variables, 
- columns C and D contain random noise
- column E is a sum of last 10 values of B multiplied by D

In [2]:
df = pd.DataFrame({'A': np.arange(1000), 
                   'B': (np.random.rand(1000)> .5) * 1.0, 
                   'C': np.random.rand(1000), 
                   'D': np.random.rand(1000)})
df['E'] = df['B'] * df['D'] 
df['E'] = np.cumsum(df['E'])
df.loc[10:, 'E'] -= np.array(df['E'][:-10])
print(df.head(20))

     A    B         C         D         E
0    0  1.0  0.262577  0.916297  0.916297
1    1  1.0  0.568105  0.789437  1.705734
2    2  0.0  0.272038  0.564149  1.705734
3    3  1.0  0.906101  0.455046  2.160780
4    4  1.0  0.324807  0.245530  2.406309
5    5  1.0  0.296518  0.520783  2.927093
6    6  0.0  0.492555  0.256756  2.927093
7    7  1.0  0.367418  0.185683  3.112776
8    8  1.0  0.045470  0.635562  3.748338
9    9  0.0  0.403797  0.548697  3.748338
10  10  0.0  0.295046  0.314787  2.832041
11  11  1.0  0.016646  0.478048  2.520652
12  12  0.0  0.529932  0.126265  2.520652
13  13  1.0  0.532256  0.854418  2.920024
14  14  0.0  0.597761  0.912888  2.674495
15  15  0.0  0.506280  0.390183  2.153711
16  16  0.0  0.350338  0.171528  2.153711
17  17  0.0  0.377504  0.146421  1.968028
18  18  0.0  0.816440  0.496657  1.332466
19  19  0.0  0.530889  0.305724  1.332466


Saving data frame to csv. Note that csv format is required by the model to read from.

In [3]:
dataset_file = os.path.join(WDIR, 'data', 'example1.csv')
df.to_csv(dataset_file)

Speifying parameters for training. For each of the keyword parameters we define a list of values for grid search.

We want to train models that predict column A given B, C and D, and A and E, given B, C and D.  

In [9]:
param_dict = dict(
    # input parameters
    input_column_names = [['B', 'C', 'D']],            # input columns 
    target_column_names = [['E'], ['A', 'E']],         # target columns
    diff_column_names = [[]],                          # columns to take first difference of   
    verbose = [2],                  # verbosity
    train_share = [(.8, .9, 1.)],   # delimeters of the training and validation shares
    input_length = [20],            # input length (1 - stateful lstm)
    output_length = [1],            # no. of timesteps to predict (only 1 impelemented)
    batch_size = [16],              # batch size
    objective=['regr'],             # only 'regr' (regression) implemented
    #training_parameters
    patience = [5],                 # no. of epoch after which learning rate will decrease if no improvement
    reduce_nb = [2],                # no. of learning rate reductions
    lr = [.01],                     # initial learning rate
    clipnorm = [1.0],               # max gradient norm
    #model parameters
    norm = [10],                    # max norm for fully connected top layer's weights
    filters = [8],                  # no. of convolutional filters per layer
    act = ['leakyrelu'],            # activation ('linear', 'relu', 'sigmoid', 'tanh', 'leakyrelu')
    kernelsize = [[1, 3], 1, 3],    # kernel size (if list of ints passed kernel size changes successively in consecutive layers)
    layers_no = [{'sigs': 5, 'offs': 1}],                # no. of layers for significance and offset sub-networks             
    architecture = [{'softmax': True, 'lambda': False}], # final activation: lambda=True indicates softplus   
    nonnegative = [False],          # if True, apply only nonnegative weights at the top layer
    connection_freq = [2],          # vertical connection frequency for ResNet
    aux_weight = [0.1],             # auxilllary loss weight
    shared_final_weights = [False], # if True, same weights of timesteps for all dimentions are trained
    resnet = [False],               # if True, adds vertical connections
)

Model class import. Other benchmark models such as LSTM, CNN or ResNet can be found in nnts.models module.

In [10]:
from nnts.models import SOCNN

Running the grid seach. Results will be pickled in 'nnts/results' directory.

In [11]:
save_file = os.path.join(WDIR, 'results', 'example_model.pkl')
runner = nnts.utils.ModelRunner(param_dict, [dataset_file], save_file)

results = runner.run(SOCNN.SOCNNmodel, log=False, limit=3)

Found 0 (< limit = 3) computed results for the setting.
As yet, for this configuration: success: 0, errors: 0
     batch_size: 16
      objective: regr
     kernelsize: [1, 3]
target_column_names: ['E']
      reduce_nb: 2
       patience: 5
   architecture: {'lambda': False, 'softmax': True}
connection_freq: 2
      layers_no: {'sigs': 5, 'offs': 1}
     aux_weight: 0.1
    train_share: (0.8, 0.9, 1.0)
             lr: 0.01
    nonnegative: False
input_column_names: ['B', 'C', 'D']
           norm: 10
            act: leakyrelu
       clipnorm: 1.0
  output_length: 1
         resnet: False
        verbose: 2
shared_final_weights: False
   input_length: 20
diff_column_names: []
        filters: 8
using <class 'nnts.models.SOCNN.SOCNNmodel'> to build the model
Total model parameters: 606
INFO:tensorflow:Summary name significance1/kernel:0 is illegal; using significance1/kernel_0 instead.
INFO:tensorflow:Summary name significance1/kernel:0 is illegal; using significance1/kernel_0 instead.

KeyboardInterrupt: 

Results are stored in a list of dictionaries. Each dictionary stores parameters and results (loss function values troughout consecutive epochs) that correspond to each single model fitted during training.

In [23]:
for i, r in enumerate(results[:3]):
    print('########## results[%d] ##########' % i)
    for k, v in r.items():
        print(str(k).rjust(25) + ': ' + str(v)[:80] + ' ...' * (len(str(v)) > 80 ))
    print('\n\n')

########## results[0] ##########
               batch_size: 16
    val_value_output_loss: [1.5225844144821168, 1.4680827140808106, 1.4728038549423217, 1.4350318670272828, ...
         main_output_loss: [0.99950292582313216, 1.0015199854969978, 0.99857743084430695, 0.998141956826051 ...
               layer_size: nan
                 val_loss: [1.1492438793182373, 1.1463390588760376, 1.1410159826278687, 1.1299276351928711, ...
               epoch_time: [20.58132839202881, 21.08583354949951, 21.58590078353882, 22.119659900665283, 22 ...
                test_loss: [1.1629176139831543, 1.1550909042358399, 1.147585678100586, 1.1455643892288208,  ...
                  verbose: 1
            training_time: 37.48593258857727
                 patience: 5
          connection_freq: 2.0
               aux_weight: 0.1
              train_share: (0.8, 0.9, 1.0)
              nonnegative: False
                 datetime: 2017-09-19T18:52:35.811768
                     date: 2017-09-19
             